In [0]:
!pip install pytesseract
!sudo apt-get update
!sudo apt-get install tesseract-ocr
!sudo apt-get install libtesseract-dev
!pip install deskew
import matplotlib.pyplot as plt
import numpy as np
import cv2
import pytesseract
from PIL import Image, ImageEnhance, ImageFilter
from skimage import io
from skimage.transform import rotate
from skimage.color import rgb2gray
from deskew import determine_skew
import os
from math import radians, cos, sin, asin, sqrt,pi
from pathlib import Path
from osgeo import gdal

In [0]:
!gdown https://drive.google.com/uc?id=1MjBMO_Ql7kth7VPPX6XkDXW4sr1qtjDV
!gdown https://drive.google.com/uc?id=17Tat38W_M_4yaHS_dHTf2TakNaT-jQel
!unzip StraboDependency.zip
!sh install.sh
!sudo apt-get install python3-dev
!sudo apt-get install software-properties-common
!sudo apt-get install libgeos-dev
!sudo pip3 install -r text-detection-requirements.txt
!sudo add-apt-repository -y ppa:alex-p/tesseract-ocr
!sudo apt-get update
!sudo apt install -y libtesseract-dev libleptonica-dev liblept5
!sudo apt install -y tesseract-ocr
!sudo pip3 install git+https://github.com/spatial-computing/tesserocr
!export LC_ALL=C
%cd strabo-text-recognition-deep-learning/
!wget "https://uob-my.sharepoint.com/:u:/g/personal/rs16175_bristol_ac_uk/ESYILU6Hu4dGoo3gfNr6rHUB75nM6zZjcTvie_3gbY74iA?download=1" --output-document "MapData.zip"
!unzip MapData.zip 

In [0]:
!rm -r test

In [5]:
%cd strabo-text-recognition-deep-learning/

/content/strabo-text-recognition-deep-learning


In [0]:
!unzip text_detection.zip #Rob's OCR base points
# !wget "https://uob-my.sharepoint.com/:u:/g/personal/rs16175_bristol_ac_uk/ESYILU6Hu4dGoo3gfNr6rHUB75nM6zZjcTvie_3gbY74iA?download=1" --output-document "MapData.zip"
# !unzip MapData.zip 

In [0]:
city = "Burnley"
dataBasePath = "Sample/Raw/"+city

def findAllFileIds():
  prjFiles = list(filter(lambda x: x.endswith(".prj"),os.listdir(dataBasePath)))
  return list(map(lambda x: Path(x).stem,prjFiles))

def setup():
  fileIds = findAllFileIds()
  for name in fileIds:
    gdal.Translate(os.getcwd()+"/"+dataBasePath+"/"+name + '.png',
               os.getcwd()+"/"+dataBasePath+"/"+name + '.tif',
               options="-of PNG")

# setup()

In [0]:

def allImagesToText():
  maps = findAllFileIds()
  for tile in maps:
      print(tile)
      textInTile = imageToText(tile)
      with open("LatLonTextOCR/"+city+"/"+tile+'.txt','a') as f:
        for item in textInTile:
            f.write(str(item[0])+","+str(item[1])+","+item[2].replace('\n', ' ').replace('\r', '')+"\n")

def imageToText(imageId):
  with open("content/MapTD/data/output/"+city+"/"+imageId+".txt", 'r') as f:
    lines = f.readlines()
  
  os.mkdir(imageId)
  latLonTuples = []
  x = len(lines)
  counter = 0
  for line in lines:
    try:
      ctr, w, h, t = retrieveImageRequiredMetrics(line)
      image = cv2.imread(dataBasePath+"/"+imageId+".png")
      image = subimage(image, center=ctr, theta=t, width=w, height=h)
      image = deskewImage(h,w,image)
      image.save(str(imageId)+"/"+str(counter)+".png")
      lat,lon = mapImagePixelToLatLon(ctr,imageId)
      with open(str(imageId)+"/"+str(counter)+".txt",'a') as f:
        f.write(str(lat)+","+str(lon)+"\n")
    except Exception as e:
      print(e)
      pass
    if counter % 10 == 0:
      print("Line " + str(counter) + " of " + str(x))
    counter += 1
  return latLonTuples

def subimage(image, center, theta, width, height):
   shape = ( image.shape[1], image.shape[0] )
   matrix = cv2.getRotationMatrix2D( center=center, angle=theta, scale=1 )
   image = cv2.warpAffine( src=image, M=matrix, dsize=shape )
   x = int( center[0] - width/2  )
   y = int( center[1] - height/2 )
   image = image[ y-20:y+height+20, x-20:x+width+20 ]
   return image

def retrieveImageRequiredMetrics(textline):
  pointsList = list(map(lambda x: int(float(x)), textline.split(",")[:-2]))
  xs = pointsList[0::2]
  ys = pointsList[1::2]
  coords = np.array(list(zip(xs,ys)))
  (centerX,centerY), (width,height),theta = cv2.minAreaRect(coords)
  return (int(centerX)+1,int(centerY)+1), int(width)+1, int(height)+1, int(theta)

def textFromImage(im):
  return "Hi"
  text = pytesseract.image_to_string(im)
  return text

def deskewImage(h,w,image):
  if h > w:
      image = np.rot90(image)
    
  # grayscale = rgb2gray(image)
  # angle = determine_skew(grayscale)
  # image = rotate(image, angle,resize=True,cval=1)
  image = Image.fromarray(image)
  return image

In [0]:
def mapImagePixelToLatLon(coord,imageId):
  targetFile = dataBasePath+"/"+imageId
  targetFileGeo = targetFile + ".tfw"
  lat,lon = getBaseLatLon()
  Xm, Ym = convertPixelToMetreOffset(coord,targetFileGeo)
  newLatFromBase, newLonFromBase = transformLatLonByXMetres(lat,lon,Ym,Xm)
  return (newLatFromBase, newLonFromBase)

def getBaseLatLon():
  return 53.9153862, -2.6224470139

def convertPixelToMetreOffset(coord,targetFileGeo):
  pixelX, pixelY = coord
  with open(targetFileGeo, 'rb') as f:
    lines = f.readlines()
    lonOffsetFromBase = float(lines[-1].strip())
    latOffsetFromBase = float(lines[-2].strip())
    yPixelsToMetres = 1 / float(lines[-3].strip())
    xPixelsToMetres = 1 / float(lines[0].strip())

    metreOffsetOnY = pixelY / yPixelsToMetres
    metreOffsetOnX = pixelX / xPixelsToMetres

    return (metreOffsetOnX+latOffsetFromBase,metreOffsetOnY+lonOffsetFromBase)
  
def transformLatLonByXMetres(lat,lon,dn,de):
    R=6377563.396 #from their files
    dLat = dn/R
    dLon = de/(R*cos(pi*lat/180))
    latO = lat + dLat * 180/pi
    lonO = lon + dLon * 180/pi

    return (latO,lonO)

In [0]:
import shutil
import json

def getAllTextForCity():
  textTuples = []
  maps = findAllFileIds()
  shutil.rmtree("static/results/test2")
  exceptedFiles = []
  for tile in maps:
    try:
      textInTile = retrieveTextWLatLonsFromImage(tile)
    except Exception as e:
      print(e)
      exceptedFiles.append(tile)
    shutil.rmtree("static/results/test2")
  return textTuples, exceptedFiles

def retrieveTextWLatLonsFromImage(imageId):
  numFiles = len(os.listdir(imageId))
  for i in range(numFiles):
    targetFile = str(imageId)+"/"+str(i)
    targetFileImage = targetFile + ".png"
    !python3 run_command_line.py --image=$targetFileImage
    text = retrieveModelOutputStrabo(targetFileImage)
    
    with open(str(imageId)+"/"+str(i)+".txt",'r') as f:
      latLons = f.read()[:-1]
    with open("Burnley/"+imageId+".txt",'a') as f:
      f.write(latLons+","+text)

  return text

def retrieveModelOutputStrabo(target):
  with open(target,'rb') as f:
    parsed = (json.load(f))
    strings = " ".join(list(map(lambda x: x[['NameAfterDictionary'].replace('\n', ' ').replace('\r', '')],parsed['features'])))
    return strings+"\n"

getAllTextForCity()

In [0]:
allImagesToText()

1909064020211
Line 0 of 469
Line 10 of 469
Line 20 of 469
Line 30 of 469
Line 40 of 469
Line 50 of 469
Line 60 of 469
Line 70 of 469
Line 80 of 469
Line 90 of 469
Line 100 of 469
Line 110 of 469
Line 120 of 469
Line 130 of 469
tile cannot extend outside image
Line 140 of 469
Line 150 of 469
Line 160 of 469
Line 170 of 469
Line 180 of 469
Line 190 of 469
Line 200 of 469
Line 210 of 469
Line 220 of 469
Line 230 of 469
Line 240 of 469
Line 250 of 469
Line 260 of 469
Line 270 of 469
Line 280 of 469
Line 290 of 469
Line 300 of 469
Line 310 of 469
Line 320 of 469
Line 330 of 469
Line 340 of 469
Line 350 of 469
Line 360 of 469
Line 370 of 469
Line 380 of 469
Line 390 of 469
Line 400 of 469
Line 410 of 469
Line 420 of 469
tile cannot extend outside image
Line 430 of 469
Line 440 of 469
Line 450 of 469
Line 460 of 469
1909064010191
Line 0 of 213
Line 10 of 213
Line 20 of 213
Line 30 of 213
Line 40 of 213
Line 50 of 213
Line 60 of 213
Line 70 of 213
